### 210. Course Schedule II

**時間複雜度: $O( V + E )$**  
**空間複雜度: $O( V + E )$**

- V: 節點數（課程數）
- E: 邊數（先修條件數）

### bfs

In [ ]:
from typing import List
from collections import deque

class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        # 建立鄰接表和入度表
        graph = [[] for _ in range(numCourses)] # space: O(E)
        indegree = [0] * numCourses # space: O(V)
        
        # 建立鄰接表和計算每個節點的入度
        for course, prereq in prerequisites:
            graph[prereq].append(course)
            indegree[course] += 1
            
        # 將所有入度為0的課程加入隊列
        queue = deque() # space: O(V)
        for i in range(numCourses):
            if indegree[i] == 0:
                queue.append(i)
                
        result = [] # space: O(V)
        
        # BFS
        while queue:
            curr = queue.popleft()
            result.append(curr)
            
            # 將當前課程的所有後續課程的入度減1
            for next_course in graph[curr]:
                indegree[next_course] -= 1
                # 如果入度變為0，表示可以修這門課了
                if indegree[next_course] == 0:
                    queue.append(next_course)
                    
        # 如果無法完成所有課程（存在環），返回空列表
        return result if len(result) == numCourses else []

In [ ]:
# empty (沒有先修條件，直接返回課程順序)
numCourses = 1
prerequisites = []
Solution().findOrder(numCourses, prerequisites) # [0]

graph={}

----------------------------------------------------------------------------------------------------

course=0, visited=[0]
visited[0] = 1
visited=[1]
visited[0] = 2
visited=[2]
result=[0] -> return True
course=0, status=True
____________________________________________________________________________________________________


[0]

In [ ]:
# cycle (有環，已經訪問過的課程，再次訪問，表示有環，無法完成所有課程。ex: 1 -> 0 -> 1)
numCourses = 2
prerequisites = [[1, 0], [0, 1]]
Solution().findOrder(numCourses, prerequisites) # []

graph={0: [1], 1: [0]}

----------------------------------------------------------------------------------------------------

course=0, visited=[0, 0]
visited[0] = 1
visited=[1, 0]

course=0 -> next_course=1
----------------------------------------------------------------------------------------------------

course=1, visited=[1, 0]
visited[1] = 1
visited=[1, 1]

course=1 -> next_course=0
----------------------------------------------------------------------------------------------------

course=0, visited=[1, 1]
visited[0] == 1 -> return False

course=1 -> next_course=0, status=False
return False

course=0 -> next_course=1, status=False
return False
course=0, status=False
____________________________________________________________________________________________________
return []


[]

In [ ]:
# normal (沒有環，可以完成所有課程)
numCourses = 4
prerequisites = [[1, 0], [2, 0], [3, 1], [3, 2]]
Solution().findOrder(numCourses, prerequisites) # [0, 1, 2, 3]

graph={0: [1, 2], 1: [3], 2: [3]}

----------------------------------------------------------------------------------------------------

course=0, visited=[0, 0, 0, 0]
visited[0] = 1
visited=[1, 0, 0, 0]

course=0 -> next_course=1
----------------------------------------------------------------------------------------------------

course=1, visited=[1, 0, 0, 0]
visited[1] = 1
visited=[1, 1, 0, 0]

course=1 -> next_course=3
----------------------------------------------------------------------------------------------------

course=3, visited=[1, 1, 0, 0]
visited[3] = 1
visited=[1, 1, 0, 1]
visited[3] = 2
visited=[1, 1, 0, 2]
result=[3] -> return True

course=1 -> next_course=3, status=True
visited[1] = 2
visited=[1, 2, 0, 2]
result=[3, 1] -> return True

course=0 -> next_course=1, status=True

course=0 -> next_course=2
----------------------------------------------------------------------------------------------------

course=2, visited=[1, 2, 0, 2]
visited[2] = 1
visited=[1, 2, 1, 2]



[0, 2, 1, 3]

### dfs

In [33]:
from typing import List

class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        # 建立鄰接表來表示課程之間的依賴關係圖
        # key: 先修課程，value: 後續可以修的課程列表
        graph = {} # space: O(E)，E: 邊數（先修條件數）
        for course, prereq in prerequisites:
            graph[prereq] = graph.get(prereq, []) + [course]
        print(f"{graph=}\n")

        # 用來追蹤每個課程的狀態
        # 0: 未訪問, 1: 正在訪問（用於檢測環）, 2: 已完成訪問
        visited = [0] * numCourses # space: O(V)，V: 節點數（課程數）
        # 存儲最終的課程修習順序
        result = [] # space: O(V)，V: 節點數（課程數）

        def dfs(course):
            print("-"*100)
            print(f"\n{course=}, {visited=}")

            # 如果當前課程正在被訪問，表示存在環（循環依賴）
            if visited[course] == 1:
                print(f"visited[{course}] == 1 -> return False")
                return False
            
            # 如果課程已經完成訪問，不需要再次處理
            if visited[course] == 2:
                print(f"visited[{course}] == 2 -> return True")
                return True

            # 標記當前課程為正在訪問
            visited[course] = 1
            print(f"visited[{course}] = 1")
            print(f"{visited=}")

            # 遍歷當前課程的所有後續課程
            for next_course in graph.get(course, []): # time: O(E)，E: 邊數（先修條件數）
                print(f"\n{course=} -> {next_course=}")
                status = dfs(next_course)
                print(f"\n{course=} -> {next_course=}, {status=}")
                # 如果在遍歷後續課程時發現環，返回 False
                if not status:
                    print(f"return False")
                    return False

            # 標記當前課程為已完成訪問
            visited[course] = 2
            # 將當前課程加入結果列表
            # （由於最後會反轉結果，所以這裡是按照完成順序加入）
            result.append(course)

            print(f"visited[{course}] = 2")
            print(f"{visited=}")
            print(f"{result=} -> return True")

            return True

        # 對每個課程進行 DFS
        for course in range(numCourses): # time: O(V)，V: 節點數（課程數）
            status = dfs(course)
            print(f"{course=}, {status=}")
            print("_"*100)
            
            # 如果發現環，返回空列表表示無法完成所有課程
            if not status:
                print(f"return []")
                return []

        # 反轉結果得到正確的修課順序 （因為 DFS 是從後往前加入的）
        return result[::-1]

In [34]:
# empty (沒有先修條件，直接返回課程順序)
numCourses = 1
prerequisites = []
Solution().findOrder(numCourses, prerequisites) # [0]

graph={}

----------------------------------------------------------------------------------------------------

course=0, visited=[0]
visited[0] = 1
visited=[1]
visited[0] = 2
visited=[2]
result=[0] -> return True
course=0, status=True
____________________________________________________________________________________________________


[0]

In [35]:
# cycle (有環，已經訪問過的課程，再次訪問，表示有環，無法完成所有課程。ex: 1 -> 0 -> 1)
numCourses = 2
prerequisites = [[1, 0], [0, 1]]
Solution().findOrder(numCourses, prerequisites) # []

graph={0: [1], 1: [0]}

----------------------------------------------------------------------------------------------------

course=0, visited=[0, 0]
visited[0] = 1
visited=[1, 0]

course=0 -> next_course=1
----------------------------------------------------------------------------------------------------

course=1, visited=[1, 0]
visited[1] = 1
visited=[1, 1]

course=1 -> next_course=0
----------------------------------------------------------------------------------------------------

course=0, visited=[1, 1]
visited[0] == 1 -> return False

course=1 -> next_course=0, status=False
return False

course=0 -> next_course=1, status=False
return False
course=0, status=False
____________________________________________________________________________________________________
return []


[]

In [36]:
# normal (沒有環，可以完成所有課程)
numCourses = 4
prerequisites = [[1, 0], [2, 0], [3, 1], [3, 2]]
Solution().findOrder(numCourses, prerequisites) # [0, 1, 2, 3]

graph={0: [1, 2], 1: [3], 2: [3]}

----------------------------------------------------------------------------------------------------

course=0, visited=[0, 0, 0, 0]
visited[0] = 1
visited=[1, 0, 0, 0]

course=0 -> next_course=1
----------------------------------------------------------------------------------------------------

course=1, visited=[1, 0, 0, 0]
visited[1] = 1
visited=[1, 1, 0, 0]

course=1 -> next_course=3
----------------------------------------------------------------------------------------------------

course=3, visited=[1, 1, 0, 0]
visited[3] = 1
visited=[1, 1, 0, 1]
visited[3] = 2
visited=[1, 1, 0, 2]
result=[3] -> return True

course=1 -> next_course=3, status=True
visited[1] = 2
visited=[1, 2, 0, 2]
result=[3, 1] -> return True

course=0 -> next_course=1, status=True

course=0 -> next_course=2
----------------------------------------------------------------------------------------------------

course=2, visited=[1, 2, 0, 2]
visited[2] = 1
visited=[1, 2, 1, 2]



[0, 2, 1, 3]